# 4.1: Task 1 & Task 2 

# CHƯƠNG 4 – PHÂN TÍCH DỮ LIỆU BẰNG PYTHON

## 4.1. Thống kê mô tả

Notebook này thực hiện:
- Task 1 (4.1.1): Thống kê tần suất đặc điểm mẫu (địa điểm, loại hình kinh doanh, tần suất sử dụng dịch vụ).
- Task 2 (4.1.2): Thống kê mô tả thang đo (các biến quan sát RE, OU, PR, MA, ISR, P, CS).

Dữ liệu đầu vào: file CSV chứa các cột  
`timestamp,email,company,location,business_type,position,experience,used_service,usage_frequency,RE1,...,CS5`.


In [1]:
import pandas as pd
import numpy as np

# ❗ ĐỔI đường dẫn file cho đúng tên file của bạn
csv_path = "data_labeled.csv"

# Đọc dữ liệu (UTF-8 hoặc UTF-8-SIG tuỳ file)
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# Xem 5 dòng đầu để kiểm tra
df.head()


,timestamp,email,company,location,business_type,position,experience,used_service,usage_frequency,RE1,...,P2,P3,P4,P5,P6,CS1,CS2,CS3,CS4,CS5
0,11/6/2025 9:18:52,ducthach120990@gmail.com,Ecu worldwide,Kho 5,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,4,...,3,3,3,3,3,4,4,4,4,4
1,11/6/2025 9:20:20,hiepanctancang@gmail.com,TNHH DỊCH VỤ và VẬN TẢI THẾ GIỚI CHÍNH PHƯƠNG,"Tòa nhà Sky Center, CH2A, Tầng trệt, Block C, ...",Công ty xuất nhập khẩu,Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,5,5,4,5,5,5,5,5
2,11/6/2025 9:25:51,huunguyen1801@gmail.com,Shipco Transport Việt Nam,01 đinh lễ quận 4,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,5,5,5,5,5,5,5,5
3,11/6/2025 13:42:17,nguyenketoan1@gmail.com,Melody,Số 1 nguyễn văn đậu,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,1 - 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,4,5,4,5,5,5,5,5
4,11/6/2025 13:42:50,duongtran.ab91@gmail.com,Ecu worldwide,Tp hochiminh,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,4,4,4,4,4,4,4,4,4,4


### Định nghĩa thang đo (SCALES) cho các biến tiềm ẩn

- RE: Resource (Nguồn lực) – RE1–RE5  
- OU: Outcome (Kết quả) – OU1–OU5  
- PR: Process (Quy trình) – PR1–PR4  
- MA: Management (Quản lý) – MA1–MA6  
- HA: Image & Social responsibility (Hình ảnh & TNXH) – ISR1–ISR3  
- P: Price (Giá cả) – P1–P6  
- CS: Customer satisfaction (Hài lòng khách hàng – biến phụ thuộc) – CS1–CS5


In [2]:
SCALES = {
    "RE": ["RE1", "RE2", "RE3", "RE4", "RE5"],                # Resource
    "OU": ["OU1", "OU2", "OU3", "OU4", "OU5"],                # Outcome
    "PR": ["PR1", "PR2", "PR3", "PR4"],                       # Process
    "MA": ["MA1", "MA2", "MA3", "MA4", "MA5", "MA6"],        # Management
    "HA": ["ISR1", "ISR2", "ISR3"],                          # Image & Social responsibility
    "P":  ["P1", "P2", "P3", "P4", "P5", "P6"],              # Price
    "CS": ["CS1", "CS2", "CS3", "CS4", "CS5"],               # Customer satisfaction (DV)
}

SCALES


{'RE': ['RE1', 'RE2', 'RE3', 'RE4', 'RE5'],
 'OU': ['OU1', 'OU2', 'OU3', 'OU4', 'OU5'],
 'PR': ['PR1', 'PR2', 'PR3', 'PR4'],
 'MA': ['MA1', 'MA2', 'MA3', 'MA4', 'MA5', 'MA6'],
 'HA': ['ISR1', 'ISR2', 'ISR3'],
 'P': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6'],
 'CS': ['CS1', 'CS2', 'CS3', 'CS4', 'CS5']}

## 4.1.1. Task 1 – Thống kê tần suất đặc điểm của mẫu nghiên cứu

Mục tiêu:
- Tính **tần suất (Frequency)** và **tỷ lệ phần trăm (%)** cho các biến mô tả mẫu:
  - `location` – Địa điểm đăng ký kinh doanh
  - `business_type` – Loại hình kinh doanh
  - `usage_frequency` – Tần suất sử dụng dịch vụ

Kết quả sẽ dùng để viết Bảng 4.1 trong luận văn.


In [3]:
def frequency_table(series: pd.Series, var_label: str = None, normalize: bool = True):
    """
    Tạo bảng tần suất & tỷ lệ (%) cho một biến phân loại.
    - series: df['tên_cột']
    - var_label: tên hiển thị của biến (ví dụ 'Địa điểm đăng ký kinh doanh')
    """
    s = series.dropna()
    
    freq = s.value_counts(dropna=False)
    perc = s.value_counts(normalize=True, dropna=False) * 100
    
    table = pd.DataFrame({
        "Category": freq.index,
        "Frequency": freq.values,
        "Percent": perc.round(1).values
    })
    
    if var_label:
        table.insert(0, "Variable", var_label)
    return table


In [4]:
freq_location = frequency_table(
    df["location"],
    var_label="Địa điểm đăng ký kinh doanh"
)

freq_location


,Variable,Category,Frequency,Percent
0,Địa điểm đăng ký kinh doanh,Tphcm,13,8.0
1,Địa điểm đăng ký kinh doanh,Dong nai,8,4.9
2,Địa điểm đăng ký kinh doanh,TPHCM,6,3.7
3,Địa điểm đăng ký kinh doanh,Hồ Chí Minh,6,3.7
4,Địa điểm đăng ký kinh doanh,Lâm đồng,4,2.5
...,...,...,...,...
108,Địa điểm đăng ký kinh doanh,Thủ đức tp hcm,1,0.6
109,Địa điểm đăng ký kinh doanh,Phu an hcm,1,0.6
110,Địa điểm đăng ký kinh doanh,Hồ Chí Minh,1,0.6
111,Địa điểm đăng ký kinh doanh,TP Ho Chi Minh,1,0.6


In [5]:
freq_business_type = frequency_table(
    df["business_type"],
    var_label="Loại hình kinh doanh"
)

freq_business_type


,Variable,Category,Frequency,Percent
0,Loại hình kinh doanh,Công ty xuất nhập khẩu,112,69.1
1,Loại hình kinh doanh,Công ty giao nhận vận tải (Forwader),42,25.9
2,Loại hình kinh doanh,Công ty chuyên sản xuất/ cung cấp dịch vụ,4,2.5
3,Loại hình kinh doanh,Hãng tàu,2,1.2
4,Loại hình kinh doanh,Sản xuất,1,0.6
5,Loại hình kinh doanh,Khác,1,0.6


Nếu muốn gộp các bảng tần suất lại thành một file (để copy vào Word/Excel),
có thể nối các bảng lại hoặc xuất ra nhiều sheet Excel.


In [6]:
freq_usage = frequency_table(
    df["usage_frequency"],
    var_label="Tần suất sử dụng dịch vụ"
)

freq_usage


,Variable,Category,Frequency,Percent
0,Tần suất sử dụng dịch vụ,Trên 3 lần/tháng,134,82.7
1,Tần suất sử dụng dịch vụ,1 - 3 lần/tháng,28,17.3


In [7]:
with pd.ExcelWriter("tab4_1_frequencies.xlsx", engine="xlsxwriter") as writer:
    freq_location.to_excel(writer, sheet_name="location", index=False)
    freq_business_type.to_excel(writer, sheet_name="business_type", index=False)
    freq_usage.to_excel(writer, sheet_name="usage_frequency", index=False)

"Đã lưu file tab4_1_frequencies.xlsx"


'Đã lưu file tab4_1_frequencies.xlsx'

## 4.1.2. Task 2 – Thống kê mô tả thang đo

Mục tiêu:
- Tính **Giá trị nhỏ nhất, lớn nhất, trung bình, độ lệch chuẩn** cho **từng biến quan sát** (RE1–RE5, OU1–OU5, …, CS1–CS5).
- Kết quả tương tự Bảng 4.2 trong luận văn mẫu.

Ta sẽ làm 2 bước:
1. Thống kê mô tả cho **tất cả biến quan sát**.
2. (Tuỳ chọn) Thống kê mô tả **theo từng nhân tố** (RE, OU, PR, MA, HA, P, CS) bằng cách lấy điểm trung bình các biến trong cùng thang đo cho mỗi đáp viên.


In [8]:
# Lấy toàn bộ tên biến quan sát từ SCALES
item_cols = [item for items in SCALES.values() for item in items]

# Kiểm tra các cột này có đủ trong df không
missing = [c for c in item_cols if c not in df.columns]
print("Số biến quan sát:", len(item_cols))
print("Thiếu cột nào không?", missing)


Số biến quan sát: 34
Thiếu cột nào không? []


In [9]:
# Mô tả thống kê các biến quan sát
desc_items = df[item_cols].agg(["count", "min", "max", "mean", "std"]).T

# Đổi tên cột cho giống cách trình bày trong luận văn
desc_items = desc_items.rename(columns={
    "count": "N",
    "min": "Giá trị nhỏ nhất",
    "max": "Giá trị lớn nhất",
    "mean": "Giá trị trung bình",
    "std": "Độ lệch chuẩn",
})

# Làm tròn 3 chữ số thập phân
desc_items["Giá trị trung bình"] = desc_items["Giá trị trung bình"].round(3)
desc_items["Độ lệch chuẩn"] = desc_items["Độ lệch chuẩn"].round(3)

desc_items


,N,Giá trị nhỏ nhất,Giá trị lớn nhất,Giá trị trung bình,Độ lệch chuẩn
RE1,162.0,1.0,5.0,4.117,0.644
RE2,162.0,2.0,5.0,4.025,0.747
RE3,162.0,2.0,5.0,4.080,0.739
RE4,162.0,1.0,5.0,4.241,0.703
RE5,162.0,1.0,5.0,4.080,0.804
OU1,162.0,1.0,5.0,4.000,0.772
OU2,162.0,2.0,5.0,4.111,0.696
OU3,162.0,1.0,5.0,4.173,0.727
OU4,162.0,2.0,5.0,4.173,0.674
OU5,162.0,1.0,5.0,4.142,0.730


In [10]:
desc_items.to_excel("tab4_2_descriptive_items.xlsx", sheet_name="items", index=True)
"Đã lưu file tab4_2_descriptive_items.xlsx"


'Đã lưu file tab4_2_descriptive_items.xlsx'

### (Tuỳ chọn) Task 2b – Thống kê mô tả theo từng nhân tố

- Tạo điểm trung bình cho từng nhân tố (ví dụ: RE_mean = trung bình của RE1–RE5 trên mỗi đáp viên).
- Sau đó tính N, min, max, mean, std cho từng nhân tố.
- Dùng để viết đoạn nhận xét tổng quát:  
  “Theo thứ tự giảm dần: Process (PR), Outcome (OU), …, Price (P), Customer satisfaction (CS) đều ở mức khá trở lên…”


In [11]:
# Tạo DataFrame chứa điểm trung bình của từng nhân tố cho mỗi đáp viên
factor_scores = pd.DataFrame(index=df.index)

for factor, cols in SCALES.items():
    factor_scores[factor] = df[cols].mean(axis=1)

# Thống kê mô tả cho từng nhân tố
desc_factors = factor_scores.agg(["count", "min", "max", "mean", "std"]).T

desc_factors = desc_factors.rename(columns={
    "count": "N",
    "min": "Giá trị nhỏ nhất",
    "max": "Giá trị lớn nhất",
    "mean": "Giá trị trung bình",
    "std": "Độ lệch chuẩn",
})

desc_factors["Giá trị trung bình"] = desc_factors["Giá trị trung bình"].round(3)
desc_factors["Độ lệch chuẩn"] = desc_factors["Độ lệch chuẩn"].round(3)

desc_factors


,N,Giá trị nhỏ nhất,Giá trị lớn nhất,Giá trị trung bình,Độ lệch chuẩn
RE,162.0,1.400000,5.0,4.109,0.569
OU,162.0,2.000000,5.0,4.120,0.565
PR,162.0,1.750000,5.0,4.131,0.579
MA,162.0,1.833333,5.0,4.156,0.541
HA,162.0,2.000000,5.0,4.179,0.564
P,162.0,1.666667,5.0,4.077,0.575
CS,162.0,2.400000,5.0,4.160,0.534


In [12]:
desc_factors.to_excel("tab4_2_descriptive_factors.xlsx", sheet_name="factors", index=True)
"Đã lưu file tab4_2_descriptive_factors.xlsx"


'Đã lưu file tab4_2_descriptive_factors.xlsx'

# 4.2.1 Đánh giá mô hình đo lường (Measurement Model)
## Task 3 & Task 4 – Python + SEMOPY

Task 3:  
- Chạy mô hình SEM để lấy Outer Loadings  
- Loại biến có loading < 0.7  
- Chạy lại vòng 2  

Task 4:  
- Tính Cronbach’s Alpha  
- Composite Reliability (CR)  
- AVE  

Thư viện sử dụng: `semopy`


In [38]:
import pandas as pd
import numpy as np
from semopy import Model, Optimizer
import semopy


In [39]:
df = pd.read_csv("data_labeled.csv", encoding="utf-8-sig")

SCALES = {
    "RE": ["RE1", "RE2", "RE3", "RE4", "RE5"], 
    "OU": ["OU1", "OU2", "OU3", "OU4", "OU5"],
    "PR": ["PR1", "PR2", "PR3", "PR4"],
    "MA": ["MA1", "MA2", "MA3", "MA4", "MA5", "MA6"],
    "HA": ["ISR1", "ISR2", "ISR3"],
    "P":  ["P1","P2","P3","P4","P5","P6"],
    "CS": ["CS1","CS2","CS3","CS4","CS5"]
}


In [40]:
model_desc = ""

# measurement model
for latent, items in SCALES.items():
    model_desc += f"{latent} =~ " + " + ".join(items) + "\n"

# structural model
model_desc += "CS ~ RE + OU + PR + MA + HA + P\n"

print(model_desc)


RE =~ RE1 + RE2 + RE3 + RE4 + RE5
OU =~ OU1 + OU2 + OU3 + OU4 + OU5
PR =~ PR1 + PR2 + PR3 + PR4
MA =~ MA1 + MA2 + MA3 + MA4 + MA5 + MA6
HA =~ ISR1 + ISR2 + ISR3
P =~ P1 + P2 + P3 + P4 + P5 + P6
CS =~ CS1 + CS2 + CS3 + CS4 + CS5
CS ~ RE + OU + PR + MA + HA + P



In [41]:
model1 = Model(model_desc)
model1.load_dataset(df)

opt = Optimizer(model1)
opt.optimize()


5.345067324992343

In [42]:
# Outer loadings = standardized factor loadings (lambda)
loadings1 = model1.inspect(std_est=True)
loadings1 = loadings1[loadings1["op"] == "~"]  # "~" = loading indicator → latent
loadings1


,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,CS,~,RE,0.000000,0.000000,0.248917,0.0,1.0
1,CS,~,OU,0.000000,0.000000,0.301074,0.0,1.0
2,CS,~,PR,0.000000,0.000000,0.28413,0.0,1.0
3,CS,~,MA,0.000000,0.000000,0.272691,0.0,1.0
4,CS,~,HA,0.000000,0.000000,0.292255,0.0,1.0
5,CS,~,P,0.000000,0.000000,0.278034,0.0,1.0
6,RE1,~,RE,1.000000,0.441868,-,-,-
7,RE2,~,RE,0.786725,0.316765,0.491758,1.599821,0.109638
8,RE3,~,RE,0.740963,0.303041,0.468895,1.580234,0.114053
9,RE4,~,RE,0.560507,0.245129,0.388011,1.444565,0.14858


In [44]:
print(loadings1.columns)


Index(['lval', 'op', 'rval', 'Estimate', 'Est. Std', 'Std. Err', 'z-value',
       'p-value'],
      dtype='object')


In [45]:
# tìm cột nào chứa "std" hoặc "Std"
std_col = [col for col in loadings1.columns if "std" in col.lower()][0]
std_col


'Est. Std'

In [46]:
low_loading1 = loadings1[loadings1[std_col] < 0.7]
low_loading1


,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,CS,~,RE,0.000000,0.000000,0.248917,0.0,1.0
1,CS,~,OU,0.000000,0.000000,0.301074,0.0,1.0
2,CS,~,PR,0.000000,0.000000,0.28413,0.0,1.0
3,CS,~,MA,0.000000,0.000000,0.272691,0.0,1.0
4,CS,~,HA,0.000000,0.000000,0.292255,0.0,1.0
5,CS,~,P,0.000000,0.000000,0.278034,0.0,1.0
6,RE1,~,RE,1.000000,0.441868,-,-,-
7,RE2,~,RE,0.786725,0.316765,0.491758,1.599821,0.109638
8,RE3,~,RE,0.740963,0.303041,0.468895,1.580234,0.114053
9,RE4,~,RE,0.560507,0.245129,0.388011,1.444565,0.14858


## ✅ Task 3 – Outer Loadings (Đánh giá mô hình đo lường)

Outer loadings (hệ số tải nhân tố) được trích xuất từ mô hình SEM (semopy).  
Các hệ số này cho biết mức độ mỗi biến quan sát phản ánh nhân tố ẩn tương ứng.

Tiêu chuẩn chấp nhận:
- Loading ≥ 0.70: tốt  
- 0.40–0.70: có thể giữ lại (nếu Cronbach Alpha, CR, AVE đạt yêu cầu)  
- < 0.40: nên loại  

Dưới đây là bảng outer loadings và danh sách các biến có loading < 0.7.


In [58]:
print("=== TASK 3 – OUTER LOADINGS (ĐÚNG CHUẨN PLS-SEM) ===")
display(loadings1)

# Xác định cột chứa loading chuẩn hóa
std_col = [col for col in loadings1.columns if "std" in col.lower()][0]

low_loading = loadings1[loadings1[std_col] < 0.7]

print("\n=== Các biến có loading < 0.7 ===")
display(low_loading)

# LƯU CHO TASK 4
outer_loadings_task3 = loadings1.copy()
low_loading_task3 = low_loading.copy()


=== TASK 3 – OUTER LOADINGS (ĐÚNG CHUẨN PLS-SEM) ===


,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,CS,~,RE,0.000000,0.000000,0.248917,0.0,1.0
1,CS,~,OU,0.000000,0.000000,0.301074,0.0,1.0
2,CS,~,PR,0.000000,0.000000,0.28413,0.0,1.0
3,CS,~,MA,0.000000,0.000000,0.272691,0.0,1.0
4,CS,~,HA,0.000000,0.000000,0.292255,0.0,1.0
5,CS,~,P,0.000000,0.000000,0.278034,0.0,1.0
6,RE1,~,RE,1.000000,0.441868,-,-,-
7,RE2,~,RE,0.786725,0.316765,0.491758,1.599821,0.109638
8,RE3,~,RE,0.740963,0.303041,0.468895,1.580234,0.114053
9,RE4,~,RE,0.560507,0.245129,0.388011,1.444565,0.14858



=== Các biến có loading < 0.7 ===


,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,CS,~,RE,0.000000,0.000000,0.248917,0.0,1.0
1,CS,~,OU,0.000000,0.000000,0.301074,0.0,1.0
2,CS,~,PR,0.000000,0.000000,0.28413,0.0,1.0
3,CS,~,MA,0.000000,0.000000,0.272691,0.0,1.0
4,CS,~,HA,0.000000,0.000000,0.292255,0.0,1.0
5,CS,~,P,0.000000,0.000000,0.278034,0.0,1.0
6,RE1,~,RE,1.000000,0.441868,-,-,-
7,RE2,~,RE,0.786725,0.316765,0.491758,1.599821,0.109638
8,RE3,~,RE,0.740963,0.303041,0.468895,1.580234,0.114053
9,RE4,~,RE,0.560507,0.245129,0.388011,1.444565,0.14858


In [60]:
# ============================
# TASK 4 – Cronbach Alpha, CR, AVE (CHUẨN)
# ============================

import pandas as pd
import numpy as np

def cronbach_alpha(df_block):
    items = df_block.values
    k = items.shape[1]
    variances = items.var(axis=0, ddof=1)
    total_variance = items.sum(axis=1).var(ddof=1)
    
    if total_variance == 0:
        return np.nan
    
    alpha = (k / (k - 1)) * (1 - variances.sum() / total_variance)
    return alpha

results = []

for latent, indicators in SCALES.items():
    # Cronbach Alpha từ dữ liệu gốc
    df_block = df[indicators]
    alpha = cronbach_alpha(df_block)
    
    # Loadings theo latent
    block = outer_loadings_task3[outer_loadings_task3["rval"] == latent]
    lambdas = block["Est. Std"].replace("-", np.nan).astype(float).dropna().values
    
    # CR
    sum_l = lambdas.sum()
    CR = (sum_l**2) / (sum_l**2 + np.sum(1 - lambdas**2))
    
    # AVE
    AVE = np.mean(lambdas**2)
    
    results.append([latent, round(alpha, 4), round(CR, 4), round(AVE, 4)])

task4_results = pd.DataFrame(results, columns=["Construct", "Cronbach Alpha", "CR", "AVE"])
display(task4_results)


,Construct,Cronbach Alpha,CR,AVE
0,RE,0.8390,0.3137,0.0872
1,OU,0.8439,0.2022,0.0522
2,PR,0.8070,0.2207,0.0711
3,MA,0.8750,0.2612,0.0590
4,HA,0.7609,0.1898,0.0792
5,P,0.8888,0.2484,0.0560
6,CS,0.8413,0.2384,0.0648


In [61]:
df.head()
df.describe()
df[SCALES["OU"]].corr()

,OU1,OU2,OU3,OU4,OU5
OU1,1.000000,0.520031,0.475569,0.560821,0.484997
OU2,0.520031,1.000000,0.526233,0.554515,0.555719
OU3,0.475569,0.526233,1.000000,0.445449,0.538634
OU4,0.560821,0.554515,0.445449,1.000000,0.555879
OU5,0.484997,0.555719,0.538634,0.555879,1.000000


In [62]:
df[SCALES["RE"]].corr()


,RE1,RE2,RE3,RE4,RE5
RE1,1.000000,0.678010,0.645592,0.513322,0.581833
RE2,0.678010,1.000000,0.626144,0.437791,0.565603
RE3,0.645592,0.626144,1.000000,0.333059,0.396943
RE4,0.513322,0.437791,0.333059,1.000000,0.394296
RE5,0.581833,0.565603,0.396943,0.394296,1.000000


In [63]:
df[SCALES["CS"]].corr()


,CS1,CS2,CS3,CS4,CS5
CS1,1.000000,0.497114,0.461245,0.449321,0.494738
CS2,0.497114,1.000000,0.494265,0.550000,0.530156
CS3,0.461245,0.494265,1.000000,0.585881,0.623613
CS4,0.449321,0.550000,0.585881,1.000000,0.484997
CS5,0.494738,0.530156,0.623613,0.484997,1.000000


In [64]:
import numpy as np
import pandas as pd

# Lấy danh sách construct theo đúng thứ tự
constructs = list(SCALES.keys())

# 1) Lấy factor scores từ semopy (latent variable scores)
factor_scores = model1.predict_factors(df)  # semopy hỗ trợ hàm này :contentReference[oaicite:0]{index=0}

# Đảm bảo cột theo đúng thứ tự construct
factor_scores = factor_scores[constructs]

# Ma trận tương quan giữa các latent variables
latent_corr = factor_scores.corr()

# 2) Lấy AVE từ kết quả Task 4
ave_map = dict(zip(task4_results["Construct"], task4_results["AVE"]))

# Căn bậc hai AVE cho từng construct
sqrt_ave = pd.Series({c: np.sqrt(ave_map[c]) for c in constructs})

# 3) Tạo bảng Fornell–Larcker: đường chéo là sqrt(AVE), ngoài đường chéo là tương quan
fornell = latent_corr.copy()
for i, c in enumerate(constructs):
    fornell.loc[c, c] = sqrt_ave[c]

print("=== Ma trận Fornell–Larcker (đường chéo = sqrt(AVE)) ===")
display(fornell.round(3))


=== Ma trận Fornell–Larcker (đường chéo = sqrt(AVE)) ===


,RE,OU,PR,MA,HA,P,CS
RE,0.295,0.797,0.642,0.788,0.672,0.714,0.799
OU,0.797,0.228,0.641,0.805,0.745,0.728,0.795
PR,0.642,0.641,0.267,0.783,0.674,0.633,0.709
MA,0.788,0.805,0.783,0.243,0.786,0.768,0.834
HA,0.672,0.745,0.674,0.786,0.281,0.716,0.750
P,0.714,0.728,0.633,0.768,0.716,0.237,0.791
CS,0.799,0.795,0.709,0.834,0.750,0.791,0.255


In [65]:
def upper_without_diag(m):
    """Lấy các phần tử phía trên đường chéo (không gồm đường chéo)."""
    idx = np.triu_indices_from(m, 1)
    vals = m[idx]
    vals = vals[~np.isnan(vals)]
    return vals

def htmt_for_pair(df, items_i, items_j):
    # Corr toàn bộ indicators của 2 construct
    data = df[items_i + items_j].corr().abs()

    # Heterotrait–heteromethod: tương quan giữa items của 2 construct khác nhau
    between = data.loc[items_i, items_j].values
    mean_between = between.flatten().mean()

    # Monotrait–heteromethod: tương quan giữa các items cùng construct
    within_i = data.loc[items_i, items_i].values
    within_j = data.loc[items_j, items_j].values

    mean_within_i = upper_without_diag(within_i).mean()
    mean_within_j = upper_without_diag(within_j).mean()

    denom = np.sqrt(mean_within_i * mean_within_j)
    return mean_between / denom

constructs = list(SCALES.keys())
htmt_mat = pd.DataFrame(
    np.eye(len(constructs)),
    index=constructs,
    columns=constructs
)

for i in range(len(constructs)):
    for j in range(i + 1, len(constructs)):
        ci, cj = constructs[i], constructs[j]
        h = htmt_for_pair(df, SCALES[ci], SCALES[cj])
        htmt_mat.loc[ci, cj] = h
        htmt_mat.loc[cj, ci] = h

print("=== Ma trận HTMT ===")
display(htmt_mat.round(3))


=== Ma trận HTMT ===


,RE,OU,PR,MA,HA,P,CS
RE,1.000,0.966,0.818,0.913,0.884,0.829,0.945
OU,0.966,1.000,0.825,0.946,0.951,0.844,0.950
PR,0.818,0.825,1.000,0.960,0.909,0.780,0.894
MA,0.913,0.946,0.960,1.000,0.996,0.864,0.966
HA,0.884,0.951,0.909,0.996,1.000,0.888,0.957
P,0.829,0.844,0.780,0.864,0.888,1.000,0.913
CS,0.945,0.950,0.894,0.966,0.957,0.913,1.000


In [66]:
import numpy as np
import pandas as pd

def compute_vif_block(X):
    """
    Tính VIF cho từng cột trong ma trận X (n x k),
    với mỗi cột lần lượt làm biến phụ thuộc, các cột còn lại là biến độc lập.
    """
    n, k = X.shape
    vifs = []
    for j in range(k):
        y = X[:, j]
        X_others = np.delete(X, j, axis=1)

        # Thêm intercept
        X_design = np.column_stack([np.ones(n), X_others])

        # Hồi quy OLS: y ~ X_others
        beta, *_ = np.linalg.lstsq(X_design, y, rcond=None)
        y_pred = X_design @ beta

        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - y.mean()) ** 2)

        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
        vif = 1.0 / (1 - r2) if (1 - r2) > 0 else np.inf
        vifs.append(vif)
    return vifs

rows = []
for latent, items in SCALES.items():
    X = df[items].values.astype(float)
    vifs = compute_vif_block(X)
    for item, vif in zip(items, vifs):
        rows.append([latent, item, vif])

outer_vif = pd.DataFrame(rows, columns=["Construct", "Indicator", "VIF"])

print("=== Outer VIF cho từng indicator ===")
display(outer_vif.round(3))


=== Outer VIF cho từng indicator ===


,Construct,Indicator,VIF
0,RE,RE1,2.640
1,RE,RE2,2.311
2,RE,RE3,1.945
3,RE,RE4,1.403
4,RE,RE5,1.671
5,OU,OU1,1.704
6,OU,OU2,1.853
7,OU,OU3,1.644
8,OU,OU4,1.845
9,OU,OU5,1.840


In [67]:
# ============================
# CHUẨN BỊ: Lấy factor scores (latent variables)
# ============================

import numpy as np
import pandas as pd

# Lấy điểm nhân tố cho tất cả latent
latent_scores = model1.predict_factors(df)

# Chỉ giữ các biến latent trong mô hình cấu trúc
latent_cols = ["RE", "OU", "PR", "MA", "HA", "P", "CS"]
latent_scores = latent_scores[latent_cols].dropna()

# Biến độc lập (predictors) và biến phụ thuộc (CS)
X_inner = latent_scores[["RE", "OU", "PR", "MA", "HA", "P"]]
y_CS    = latent_scores["CS"]

# Hàm tính R² bằng OLS (dùng cho Task 7, 8, 9)
def calc_r2(y, X):
    X_mat = np.column_stack([np.ones(len(X)), X.values])  # thêm hằng số
    beta, *_ = np.linalg.lstsq(X_mat, y.values, rcond=None)
    y_hat = X_mat @ beta
    ss_tot = np.sum((y.values - y.mean())**2)
    ss_res = np.sum((y.values - y_hat)**2)
    return 1 - ss_res / ss_tot


In [68]:
# ============================
# TASK 7 – Inner VIF
# ============================

vif_rows = []

for col in X_inner.columns:
    y_j      = X_inner[col]               # một predictor
    X_others = X_inner.drop(columns=[col])  # các predictor còn lại
    R2_j     = calc_r2(y_j, X_others)
    vif_j    = 1 / (1 - R2_j)
    vif_rows.append([col, round(vif_j, 4)])

inner_vif = pd.DataFrame(vif_rows, columns=["Construct", "Inner VIF"])
print("=== Inner VIF cho các biến độc lập trong mô hình cấu trúc ===")
display(inner_vif)


=== Inner VIF cho các biến độc lập trong mô hình cấu trúc ===


,Construct,Inner VIF
0,RE,3.4067
1,OU,3.9199
2,PR,2.6617
3,MA,5.5793
4,HA,3.0888
5,P,2.8769


In [69]:
# ============================
# TASK 8 – R² của CS (biến phụ thuộc)
# ============================

R2_CS = calc_r2(y_CS, X_inner)
print("=== R² cho biến phụ thuộc CS ===")
print(f"R²_CS = {R2_CS:.4f}")


=== R² cho biến phụ thuộc CS ===
R²_CS = 0.7947


In [70]:
# ============================
# TASK 9 – f² (effect size) cho từng biến độc lập
# ============================

f2_rows = []

# R² của mô hình đầy đủ (tất cả predictors)
R2_full = R2_CS

for col in X_inner.columns:
    X_reduced = X_inner.drop(columns=[col])   # bỏ từng predictor một
    R2_excl   = calc_r2(y_CS, X_reduced)
    f2_j      = (R2_full - R2_excl) / (1 - R2_full)
    f2_rows.append([col, round(f2_j, 4)])

f2_results = pd.DataFrame(f2_rows, columns=["Predictor", "f²"])
print("=== f² (effect size) của từng predictor lên CS ===")
display(f2_results)


=== f² (effect size) của từng predictor lên CS ===


,Predictor,f²
0,RE,0.0744
1,OU,0.0249
2,PR,0.0158
3,MA,0.0397
4,HA,0.0131
5,P,0.0955


In [73]:
lambda_CS = outer_loadings_task3[outer_loadings_task3["rval"] == "CS"][["Item", "Loading"]]
lambda_CS

KeyError: "None of [Index(['Item', 'Loading'], dtype='object')] are in the [columns]"

In [71]:
# ============================
# TASK 10 – Q² (Predictive relevance)
# ============================

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

dependent_var = "CS"       # biến phụ thuộc trong mô hình
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

y_true_all = []
y_pred_all = []

for train_idx, test_idx in kfold.split(df):
    df_train = df.iloc[train_idx].copy()
    df_test = df.iloc[test_idx].copy()

    # Fit lại model trên tập train
    model_cv = Model(model_desc)
    model_cv.load_dataset(df_train)
    opt_cv = Optimizer(model_cv)
    opt_cv.optimize()

    # Lấy latent scores
    scores = model_cv.predict_factors(df_test)

    # Dự đoán CS từ latent scores
    y_pred = model_cv.predict(df_test)[dependent_var].values
    y_true = df_test[dependent_var].values

    y_true_all.extend(y_true)
    y_pred_all.extend(y_pred)

# Tính Q²
y_true_all = np.array(y_true_all)
y_pred_all = np.array(y_pred_all)

PRESS = np.sum((y_true_all - y_pred_all)**2)
TSS = np.sum((y_true_all - np.mean(y_true_all))**2)
Q2 = 1 - PRESS/TSS

print("=== Q² Predictive relevance (Stone–Geisser) ===")
print("Q² for", dependent_var, "=", round(Q2, 4))


KeyError: 'CS'

In [72]:
# ============================
# TASK 11 – Bootstrapping
# ============================

from semopy import bootstrap

# số mẫu bootstrap (standard PLS-SEM dùng 5000)
B = 5000

boot_res = bootstrap(model1, B)

# Bootstrapped path coefficients
paths = boot_res.parameters[boot_res.parameters["op"] == "~"]

print("=== Bootstrapping – Path Coefficients ===")
display(paths[["lval", "op", "rval", "Mean", "SE", "p-value"]])

# Bootstrapped outer loadings
outer_load_boot = boot_res.parameters[boot_res.parameters["op"] == "=~"]

print("\n=== Bootstrapping – Outer Loadings ===")
display(outer_load_boot[["lval", "op", "rval", "Mean", "SE", "p-value"]])


ImportError: cannot import name 'bootstrap' from 'semopy' (c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\semopy\__init__.py)